# Transitive contradictions

In [1]:
import pickle
import random
from collections import Counter
import pandas as pd
import networkx as nx

In [2]:
stmts_df = pd.read_csv('data/stmts_by_pair_type.csv')

/Users/johnbachman/.virtualenvs/p36env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Find statements with contradictions

In [3]:
stmts_df.head()

,agA_ns,agA_id,agA_name,agB_ns,agB_id,agB_name,stmt_type,evidence_count
0,CHEBI,10043,CHEBI:10043,CHEBI,10043,CHEBI:10043,Phosphorylation,1
1,CHEBI,10043,CHEBI:10043,CHEBI,27899,CHEBI:27899,Activation,1
2,CHEBI,10043,CHEBI:10043,CHEBI,31941,CHEBI:31941,Activation,1
3,CHEBI,10043,CHEBI:10043,CHEBI,31941,CHEBI:31941,Inhibition,1
4,CHEBI,10043,CHEBI:10043,CHEBI,41879,CHEBI:41879,Inhibition,1


In [4]:
edges = []
net = nx.MultiDiGraph()
for agA_ns, agA_id, agA_name, agB_ns, agB_id, \
         agB_name, stmt_type, evidence_count in stmts_df.values:
    if not (agA_ns == 'HGNC' and agB_ns == 'HGNC'):
        continue
    if agA_name == agB_name:
        continue
    net.add_edge(agA_name, agB_name,
                 evidence_count=evidence_count,
                 stmt_type=stmt_type)

## Non-transitivity

For each edge in the network, look to see if there are relationships between the source node and the successors of the target node. If there are fewer than expected edges from source to target successor (indirect edges), then this may suggest that the edge from source to target is flawed (e.g., grounding errors).

In [12]:
edge_trans = {}
for u in net.nodes():
    u_succ = net[u]
    for v in u_succ:
        v_succ = net[v]
        if len(v_succ) > 50:
            u_succ = set(u_succ)
            v_succ =  set(v_succ)
            joint_succ = u_succ.intersection(v_succ)
            #ratio = len(joint_succ) / len(v_succ.union(u_succ))
            ratio = len(joint_succ) / len(v_succ)
            edge_trans[(u, v)] = ratio

In [13]:
sort_ratios = sorted([(k, v) for k, v in edge_trans.items()], key=lambda x: x[1], reverse=True)
zero_ratios = [t for t in sort_ratios if t[1] == 0]
random.shuffle(zero_ratios)
zero_ratios

[(('CSRP3', 'PAEP'), 0.0),
 (('ADARB1', 'ADA'), 0.0),
 (('UBR3', 'HMBS'), 0.0),
 (('ACAP2', 'BCL2L11'), 0.0),
 (('MYO1E', 'TF'), 0.0),
 (('VSX1', 'STK3'), 0.0),
 (('CHRM1', 'MEF2C'), 0.0),
 (('KREMEN2', 'EDA'), 0.0),
 (('RBPJL', 'CEL'), 0.0),
 (('CCDC115', 'EGFR'), 0.0),
 (('THRB', 'SOD1'), 0.0),
 (('DAGLA', 'SLC6A2'), 0.0),
 (('PPP5C', 'TTN'), 0.0),
 (('RHBDL2', 'RAD51'), 0.0),
 (('SMIM14', 'ESR1'), 0.0),
 (('PRR13', 'THBS1'), 0.0),
 (('NMBR', 'XPC'), 0.0),
 (('SYCP3', 'CD36'), 0.0),
 (('RALY', 'YBX1'), 0.0),
 (('ERVK-8', 'MAP1LC3B'), 0.0),
 (('NMUR2', 'MMP2'), 0.0),
 (('PLK5', 'SNCA'), 0.0),
 (('UBN1', 'MEN1'), 0.0),
 (('ARHGAP28', 'RHOA'), 0.0),
 (('HPSE2', 'USF1'), 0.0),
 (('MLNR', 'MUC1'), 0.0),
 (('DAD1', 'PCBD1'), 0.0),
 (('DCX', 'SLC19A2'), 0.0),
 (('GNGT1', 'PLCG1'), 0.0),
 (('SLC6A11', 'UGCG'), 0.0),
 (('TMEM192', 'BECN1'), 0.0),
 (('LACRT', 'FOXO3'), 0.0),
 (('KRT10', 'CBLB'), 0.0),
 (('CACNA1F', 'CACNA1A'), 0.0),
 (('ASZ1', 'GTF2I'), 0.0),
 (('PPIL2', 'GLB1'), 0.0),
 (('IFT

Of the above edges with no overlap among their joint successors, I curated the following:
1. WASHC4->SLC5A5: grounding error for SLC5A5 (NIS)
1. **RGS6->TRPC4:** correct
1. RGS6->UBC: grounding error (UBC urinary bladder cancer)
1. RGS6->CES1: grounding error (TGH "transgene high" grounded to CES1)
1. RGS6->ITGB3: from SIGNOR, possible error, RGS6 not mentioned in the paper anywhere
1. RGS9->CASP8AP2: grounding error, misgrounding from "flash"
1. RGS9->GPSM1: grounding error, "signaling" grounded to GPSM1
1. RGS9->FABP6: grounding error
1. **RGS9->PLCB3:** specific sentence is a negative result, but paper indicates a real interaction
1. RGS9->TTL: grounding error ("time to leukemia")
1. TRPC6->NRK: grounding error NRK ("normal rat kidney")
1. SPP2->PGK1: misgrounding of PRP2 (DHX16) to PGK1
1. MAP1S->CENPE: Sparser nonsense: "MAPs inhibit kinesin- and dynein-dependent mitochondrial movement along microtubules, by competing for binding to the microtubule surface"
1. **ARHGEF10->MAOA**: Correct.
1. **CSNK2A2->PDCD5**: More or less correct, but the sentence refers to "CK2 alpha subunit" rather than the specific gene, so it's likely that the specific gene has few successors.
1. **ELF2->UCHL5**: msigdb.
1. **TLL2->MSTN**: Looks legit, though study in sparrows.
1. **RORA->TLE4**: msigdb.
1. ROBO3->EPHA1: No relation in sentence (co-expression)
1. FAT2->BMP1: Misgrounding of PCP to BMP1
1. **MNX1->NONO**: Looks legit.
1. SLC10A4->IGF1: Hypothesis/negative result.
1. MTHFD1->FGR: Grounding error "fetal growth restriction"
1. **CAMK2D->TTN**: Phosphorylation from PSP--not expected to be transitive, necessarily.
1. **PTH2R->SOX9**: Correct.
1. DDX43->MCAT: Double grounding errors
1. **OPN3->BECN1:** Correct
1. ATP1B1->TWSG1: "tumor suppressor gene" misgrounded to TWSG1

In [11]:
sort_ratios

[(('ERVK-6', 'ERVK-7'), 0.9629629629629629),
 (('ERVK-6', 'ERVK-8'), 0.9629629629629629),
 (('ERVK-6', 'ERVK-19'), 0.9629629629629629),
 (('ERVK-6', 'ERVK-24'), 0.9629629629629629),
 (('ERVK-6', 'ERVK-25'), 0.9629629629629629),
 (('ERVK-7', 'ERVK-6'), 0.9629629629629629),
 (('ERVK-7', 'ERVK-8'), 0.9629629629629629),
 (('ERVK-7', 'ERVK-19'), 0.9629629629629629),
 (('ERVK-7', 'ERVK-24'), 0.9629629629629629),
 (('ERVK-7', 'ERVK-25'), 0.9629629629629629),
 (('ERVK-8', 'ERVK-6'), 0.9629629629629629),
 (('ERVK-8', 'ERVK-7'), 0.9629629629629629),
 (('ERVK-8', 'ERVK-19'), 0.9629629629629629),
 (('ERVK-8', 'ERVK-24'), 0.9629629629629629),
 (('ERVK-8', 'ERVK-25'), 0.9629629629629629),
 (('ERVK-19', 'ERVK-6'), 0.9629629629629629),
 (('ERVK-19', 'ERVK-7'), 0.9629629629629629),
 (('ERVK-19', 'ERVK-8'), 0.9629629629629629),
 (('ERVK-19', 'ERVK-24'), 0.9629629629629629),
 (('ERVK-19', 'ERVK-25'), 0.9629629629629629),
 (('ERVK-24', 'ERVK-6'), 0.9629629629629629),
 (('ERVK-24', 'ERVK-7'), 0.96296296296

What's interesting about looking at the gene pairs with high Jaccard index is that they indicate the mechanistic similarity between two genes. In some cases this happens for genes part of the same family (e.g. MAPK8 and MAPK9, JAK1 and JAK3, GRK2 and GRK3) or for a protein and a key downstream effector (e.g., EGF-EGFR, IL6-STAT3 and INSR-MTOR). In some cases there are strong links among things that are superficially dissimilar, e.g. TNF and INS, AGT, and EGF.

Notably, however, a high ratio doesn't necessarily indicate that the extraction linking the two genes is itself accurate--for example, the few sentences linking GRK2 and GRK3 were actually reading errors.

Might actually be interesting looking at the downstream targets indicated for ligands and looking at their overlap--suggests that when flattened across various contexts, many pro-inflammatory ligands can actually look very similar!

In [13]:
11/28

0.39285714285714285

## Contradictions

In [5]:
contra = []
for u in net.nodes():
    pos = set()
    neg = set()
    for v in net[u]:
        for edge_ix, data in net[u][v].items():
            if data['stmt_type'] == 'Activation':
                pos.add(v)
            elif data['stmt_type'] == 'Inhibition':
                neg.add(v)
    contra_list = [(u, n) for n in pos.intersection(neg)]
    contra.extend(contra_list)

Next, look at each pair in the contradiction, then look for the intersection of nodes downstream of both the first and second nodes, ideally where both have a single unambiguous polarity.

In [6]:
def get_edge_types(u, v):
    edge_types = set()
    for edge_ix, data in net[u][v].items():
        edge_types.add(data['stmt_type'])
    return edge_types

def get_uniq_pol_succs(u):
    u_succ = set()
    for v in net[u]:
        edge_types = get_edge_types(u, v)
        if len(set(['Activation', 'Inhibition']).intersection(edge_types)) == 1:
            u_succ.add(v)
    return u_succ

print(len(contra))



75339


In [29]:
pol_votes = {}
for ix, (u, v) in enumerate(contra):
    if u == 'KRAS':
        pass
    else:
        continue
    if ix % 1000 == 0:
        print(ix)
    u_v_votes = []
    u_succ = get_uniq_pol_succs(u)
    v_succ = get_uniq_pol_succs(v)
    joint_succ = u_succ.intersection(v_succ)
    for j in joint_succ:
        u_edge_types = get_edge_types(u, j)
        u_j_pol = 1 if 'Activation' in u_edge_types else -1
        v_edge_types = get_edge_types(v, j)
        v_j_pol = 1 if 'Activation' in v_edge_types else -1
        if u_j_pol == v_j_pol:
            u_v_votes.append('Activation')
        else:
            u_v_votes.append('Inhibition')
    pol_votes[(u, v)] = u_v_votes
            
            

In [30]:
import pickle
with open('pol_votes.pkl', 'wb') as f:
    pickle.dump(pol_votes,  f)
    
    

In [31]:
pol_votes.keys()

dict_keys([('KRAS', 'CTNNB1'), ('KRAS', 'CXCL10'), ('KRAS', 'HRAS'), ('KRAS', 'NOX1'), ('KRAS', 'GLI3'), ('KRAS', 'ROS1'), ('KRAS', 'ADM'), ('KRAS', 'NDUFAF1'), ('KRAS', 'RASGRP1'), ('KRAS', 'RB1'), ('KRAS', 'BCL2'), ('KRAS', 'SLC22A18'), ('KRAS', 'FOSL1'), ('KRAS', 'HIF1A'), ('KRAS', 'MARCH8'), ('KRAS', 'CD274'), ('KRAS', 'RREB1'), ('KRAS', 'STK3'), ('KRAS', 'NOX4'), ('KRAS', 'GLI2'), ('KRAS', 'MEFV'), ('KRAS', 'RHOA'), ('KRAS', 'DICER1'), ('KRAS', 'BRCA2'), ('KRAS', 'MIR21'), ('KRAS', 'CDH1'), ('KRAS', 'IL1B'), ('KRAS', 'TP53'), ('KRAS', 'GEM'), ('KRAS', 'STK11'), ('KRAS', 'MYC'), ('KRAS', 'IL18'), ('KRAS', 'MAP6'), ('KRAS', 'STAT3'), ('KRAS', 'CDKN1B'), ('KRAS', 'NRAS'), ('KRAS', 'IL10'), ('KRAS', 'ALK'), ('KRAS', 'CXCL1'), ('KRAS', 'TBK1'), ('KRAS', 'TPO'), ('KRAS', 'SOX9'), ('KRAS', 'GDE1'), ('KRAS', 'THBS1'), ('KRAS', 'FOXO3'), ('KRAS', 'NOTCH3'), ('KRAS', 'TERT'), ('KRAS', 'EGF'), ('KRAS', 'PEBP1'), ('KRAS', 'RASSF1'), ('KRAS', 'IMPACT'), ('KRAS', 'PDPK1'), ('KRAS', 'AKT1'), ('K

In [35]:
Counter(pol_votes[('KRAS', 'ADM')])

Counter({'Activation': 25, 'Inhibition': 7})

In [37]:
for ((u, v), pol_list) in pol_votes.items():
    ctr = Counter(pol_list)
    try:
        ratio = ctr['Inhibition'] / ctr['Activation']
    except ZeroDivisionError:
        ratio = 100
    if ratio < 0.2 or ratio > 5:
        print(u, v)

KRAS HRAS
KRAS NOX1
KRAS NDUFAF1
KRAS SLC22A18
KRAS RREB1
KRAS IL1B
KRAS IL18
KRAS NRAS
KRAS TPO
KRAS CCP110
KRAS PRSS27
